In [ ]:
# join Normal density and regression
import numpy as np
from scipy import stats
from scipy.stats import multivariate_normal as mvnorm
from matplotlib import pyplot as plt

# %matplotlib inline

In [ ]:
np.random.seed(1024)

In [ ]:
sigma = np.array([2,1,1,3]).reshape(2,2)
mux, muy = 0,0
mu = np.array([mux, muy])
rv = mvnorm(mu, sigma)

In [ ]:
sigma_inv = np.linalg.inv(sigma)
sigma_inv

In [ ]:
# principal component is the largest eigen vector
# eigenvalue decomposition:  sigma = v @ np.diag(w) @ v.T 
# note:  A @ B is matrix product
w, v = np.linalg.eig(sigma)

print(w)
print(v)
(v @ np.diag(w) @ v.T) / sigma  # check

In [ ]:
# eigen vectors
v0 = v[:,0]
v1 = v[:,1]
(v0 @ sigma) / (w[0] * v0)
(v1 @ sigma) / (w[1] * v1) 

In [ ]:
np.all((v0 @ v1) == 0)  # v0, v1 are orthogonal

In [ ]:
def conditional_pdf(x, mux, muy, sigma):
    """pdf(y|x) ~ N(mean, var)"""
    mean = muy + sigma[1,0] /sigma[0,0] * (x - mux)
    var = sigma[1,1] - sigma[1,0] /sigma[0,0] * sigma[0,1]
    return mean, var

def conditional_mean(u, mux, muy, sigma, option = 'y|x'):
    """ E(y|x) or E(x|y: regression in bivariate normal"""
    sig_xx = sigma[0,0]
    sig_yy = sigma[1,1]
    sig_xy = sigma[0,1]
    sig_yx = sigma[1,0]
    if option == 'x|y':
        fit = mux + sig_xy / sig_yy * (u - muy)
    else:
        fit = muy + sig_yx / sig_xx * (u - mux)

    return fit

In [ ]:
sigma

In [ ]:
# conditional_pdf(1, 0, 0, sigma)
gridx = np.arange(-3, 4)
[conditional_mean(u, 0, 0, sigma) for u in gridx]

In [ ]:
[conditional_mean(1,0,0,sigma, 'y|x'), conditional_mean(1,0,0,sigma, 'x|y')]

In [ ]:
x, y = np.mgrid[-6:6:.01, -6:6:.01]
pos = np.empty(x.shape + (2,)) 
pos[:,:,0] = x
pos[:,:,1] = y

In [ ]:
# plot x,y in same scale to demonstrate the othrogonality of the two principal components
plt.figure(figsize=(5,5))  
plt.contour(x,y, rv.pdf(pos), [.005])

# regression line y|x = m(x)
gridx = np.arange(-4,5)
fity = [conditional_mean(u, mux, muy, sigma, 'y|x') for u in gridx]

# regression line x/y = m(y)
gridy = np.arange(-5,6)
fitx = [conditional_mean(u, mux, muy, sigma, 'x|y') for u in gridy]

# principal component 1,2 (order by eigen value)
xy1 = np.array([v1 * u + [mux, muy] for u in np.arange(-6,6)])
xy0 = np.array([v0 * u + [mux, muy] for u in np.arange(-3,4)])


plt.plot(xy1[:,0], xy1[:,1])
plt.plot(xy0[:,0], xy0[:,1])
plt.plot(gridx, fity)
plt.plot(fitx, gridy)
plt.legend(['PC1', 'PC2', 'E(y|x)', 'E(x|y)'])

# plt.show()

In [ ]:
rv = mvnorm(mu, sigma)
sim_data = rv.rvs(200)
slope, intercept,_,_,_ = stats.linregress(sim_data)
# stats.linregress(a[:,0], a[:,1])
[slope, intercept]

In [ ]:
stats.linregress(sim_data)

In [ ]:
sigma_bar = np.cov(sim_data.T)
xbar, ybar = np.mean(sim_data, axis=0)
[xbar, ybar]

In [ ]:
# lsfit y = a + bx and E(y|x) from pdf should agree.
[slope, conditional_mean(1, 0, 0, np.cov(sim_data.T))]

In [ ]:
np.linalg.eig(sigma_bar)

In [ ]:
yfit_ls = [slope * u + intercept for u in sim_data[:,0]]
yfit_pdf = [conditional_mean(u,xbar,ybar,sigma_bar) for u in sim_data[:,0]]
#
plt.figure(figsize = (5,5))
plt.xlim(-6,6)
plt.ylim(-6,6)
plt.plot(sim_data[:,0], sim_data[:,1], '.')
plt.plot(sim_data[:,0], yfit_ls)
plt.plot(sim_data[:,0], yfit_pdf, 'b')

In [ ]:
np.cov(sim_data[:,0], sim_data[:,1])

In [ ]:
# try sklearn
from sklearn.linear_model import LinearRegression as LR

In [ ]:
x = sim_data[:,0]
y = sim_data[:,1]
model = LR()
model.fit(x[:, np.newaxis], y)
xfit = np.linspace(-4,4, 100)
yfit = model.predict(xfit[:,np.newaxis])
[model.coef_[0], model.intercept_, slope, intercept]

In [ ]:
plt.scatter(x,y)
plt.plot(xfit, yfit, 'r')
plt.show()